In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [3]:
import os
import gdown
from zipfile import ZipFile
from tqdm import tqdm

file_id = "1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf"
url = f"https://drive.google.com/uc?id={file_id}"

download_path = "./dataset/dacon_dataset.zip"
extract_path = "./dataset/dacon_dataset"

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

gdown.download(url, download_path, quiet=False)

with ZipFile(download_path, 'r') as zip_ref:
    for file in tqdm(zip_ref.namelist(), desc='Extracting files'):
        zip_ref.extract(file, extract_path)

os.remove(download_path)

print(f"File downloaded and extracted to {extract_path}")


Downloading...
From (original): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf
From (redirected): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&confirm=t&uuid=19f496fc-41e8-4478-b6a1-2508f41c8bed
To: /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/dataset/dacon_dataset.zip
100%|██████████| 3.31G/3.31G [02:07<00:00, 26.0MB/s]
Extracting files: 100%|██████████| 106708/106708 [00:24<00:00, 4304.77it/s]


File downloaded and extracted to ./dataset/dacon_dataset


# Denosing

In [2]:
import glob
import math
import os
import tempfile
import time
from typing import List, Optional, Tuple, Union

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import torch
from loguru import logger
from PIL import Image
from torch import Tensor

from df import config
from df.enhance import enhance, init_df, load_audio, save_audio
from df.io import resample

from torchaudio import AudioMetaData 

/home/hyj/anaconda3/envs/DACON_DEEPFAKE/lib/python3.11/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [3]:
os.getcwd()

'/home/hyj/ChanHyung/Audio/DACON_fake_voice_detection'

In [31]:
import pandas as pd

base_path = os.getcwd()
train_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/train.csv')
test_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/test.csv')

In [6]:
import torch
from tqdm.auto import tqdm
import shutil

import warnings
warnings.filterwarnings(action='ignore')

dir_name = 'unlabeled_data_denoising'
os.makedirs(f"./dataset/dacon_dataset/{dir_name}", exist_ok=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, df, _ = init_df(os.path.join(base_path, 'DeepFilterNet2'), config_allow_defaults=True)
model = model.to(device=device).eval()
test_x = os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data'))

for i in tqdm(range(len(test_x))):
    id = test_x[i].split('/')[-1].split('.')[0]
    try: 
        path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data', test_x[i])
        sr = config("sr", 32000, int, section="df")
        sample, meta = load_audio(path, sr)
        enhanced = enhance(model, df, sample)

        lim = torch.linspace(0.0, 1.0, int(sr * 0.15)).unsqueeze(0)
        lim = torch.cat((lim, torch.ones(1, enhanced.shape[1] - lim.shape[1])), dim=1)
        enhanced = enhanced * lim
        if meta.sample_rate != sr:
            enhanced = resample(enhanced, sr, meta.sample_rate)
            sample = resample(sample, sr, meta.sample_rate)
            sr = meta.sample_rate

        noisy_wav = os.path.join(base_path, 'noisy_wav1558.wav')
        save_audio(noisy_wav, sample, sr)
        enhanced_wav = os.path.join(base_path, 'enhanced_wav1558.wav') 
        save_audio(enhanced_wav, enhanced, sr)
        noisy_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'noisy_{id}.wav')
        shutil.move(os.path.join(base_path, 'noisy_wav1558.wav'), noisy_wav)
        enhanced_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'enhanced_{id}.wav')
        shutil.move(os.path.join(base_path, 'enhanced_wav1558.wav') ,  enhanced_wav)
    except:
        print(f"너무 짧아서 안됨: {id}")

        

2024-07-23 04:15:46 | INFO     | DF | Running on torch 2.3.0
2024-07-23 04:15:46 | INFO     | DF | Running on host cvlab


2024-07-23 04:15:46 | INFO     | DF | Loading model settings of DeepFilterNet2
2024-07-23 04:15:46 | INFO     | DF | Initializing model `deepfilternet2`


fatal: not a git repository (or any of the parent directories): .git


2024-07-23 04:15:47 | INFO     | DF | Found checkpoint /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/DeepFilterNet2/checkpoints/model_96.ckpt.best with epoch 96
2024-07-23 04:15:47 | INFO     | DF | Running on device cuda:0
2024-07-23 04:15:47 | INFO     | DF | Model loaded


  0%|          | 0/1264 [00:00<?, ?it/s]

2024-07-23 04:15:47 | WARNING  | DF | Audio sampling rate does not match model sampling rate (32000, 48000). Resampling...


# Speech Segmentation

In [7]:
audio_file_list = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_denoising', i) \
                   for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_denoising')) if "noisy" in i]

In [8]:
import torchaudio
from speechbrain.inference.VAD import VAD
import os
import json
from tqdm import tqdm

# VAD 모델 초기화
vad_model = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir="pretrained_models/vad-crdnn-libriparty")

# 오디오 파일 경로 리스트
audio_file_list = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_denoising', i) \
                   for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_denoising')) if "noisy" in i]


# 타겟 샘플링 레이트
target_sample_rate = 16000

# 음성 구간 정보를 저장할 디렉토리
output_dir = os.path.join(base_path, 'vad_bandstop_boundaries_unlabeled')
os.makedirs(output_dir, exist_ok=True)

# 오디오 파일 처리
for input_file in tqdm(audio_file_list):
    # Step 1: 오디오 파일 로드 및 샘플링 레이트 변환
    signal, original_sample_rate = torchaudio.load(input_file)
    transform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
    resampled_signal = transform(signal)
    
    # 오디오 길이 계산 (초 단위)
    audio_length = resampled_signal.size(1) / target_sample_rate
    
    # 임시 파일로 저장
    temp_output_file = os.path.join(base_path, 'temp_resampled_audio_file3.wav')
    torchaudio.save(temp_output_file, resampled_signal, target_sample_rate)
    
    # Step 2: 변환된 파일로 VAD 수행
    boundaries = vad_model.get_speech_segments(temp_output_file)
    
    # Step 3: 각 음성 구간의 시작 시간과 종료 시간을 기록
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    boundaries_list = []
    for boundary in boundaries:
        start, end = boundary[0].item(), boundary[1].item()
        boundaries_list.append({
            "start": start,
            "end": end
        })
    
    # JSON 파일로 저장
    output_data = {
        "audio_length": audio_length,
        "boundaries": boundaries_list
    }
    boundaries_filename = f"{base_filename}_boundaries.json"
    boundaries_filepath = os.path.join(output_dir, boundaries_filename)
    with open(boundaries_filepath, 'w') as f:
        json.dump(output_data, f)
    
    # print(f"Saved boundaries and audio length for {input_file} to {boundaries_filepath}")
    
    # 임시 파일 삭제
    os.remove(temp_output_file)

print("All boundaries and audio lengths have been saved.")


torchvision is not available - cannot save figures
100%|██████████| 1264/1264 [13:51<00:00,  1.52it/s]

All boundaries and audio lengths have been saved.


In [34]:
import os
import json
import librosa
import numpy as np
import soundfile as sf
from tqdm import tqdm

# 경로 설정
json_dir = os.path.join(base_path, 'vad_bandstop_boundaries_unlabeled')
audio_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data')
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_silent')


# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)


# JSON 파일 목록 가져오기
json_files = [f for f in os.listdir(json_dir) if f.endswith('_boundaries.json')]

# JSON 파일 처리
for json_file in tqdm(json_files, desc="Processing JSON files"):
    name = json_file.replace('_boundaries.json', '').replace('noisy_', '')
    
    
    # JSON 파일 경로
    json_path = os.path.join(json_dir, json_file)
    
    # 오디오 파일 경로
    audio_path = os.path.join(audio_dir, f"{name}.ogg")
    
    # JSON 파일 읽기
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # 오디오 파일 읽기
    audio_data, sr = librosa.load(audio_path, sr=None)
    
    # boundaries 가져오기
    boundaries = data.get('boundaries', [])
    
    # 간격 설정 (여기서는 0.1초를 추가로 잘라냅니다)
    margin = 0.1  # 100 milliseconds
    
    if not boundaries:
        # boundaries가 없으면 전체 오디오 데이터 사용
        cropped_audio = audio_data
    else:
        # 제외할 구간 설정
        exclude_intervals = [(max(boundary['start'] - margin, 0), min(boundary['end'] + margin, len(audio_data) / sr)) for boundary in boundaries]
        
        # 제외할 구간을 제외한 오디오 데이터 생성
        keep_samples = []
        prev_end = 0.0
        for start, end in exclude_intervals:
            if prev_end < start:
                keep_samples.append(audio_data[int(prev_end * sr):int(start * sr)])
            prev_end = end
        if prev_end * sr < len(audio_data):
            keep_samples.append(audio_data[int(prev_end * sr):])
        
        # 모든 유지할 샘플들을 합침
        cropped_audio = np.concatenate(keep_samples) if keep_samples else np.array([], dtype=audio_data.dtype)
    
    # 결과 오디오 파일 저장
    output_audio_path = os.path.join(output_dir, f"cropped_{name}.wav")
    sf.write(output_audio_path, cropped_audio, sr)


Processing JSON files: 100%|██████████| 1264/1264 [00:07<00:00, 165.60it/s]


In [35]:
import os
import soundfile as sf
import numpy as np
from tqdm import tqdm

# 경로 설정
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_silent')
merged_output_path = os.path.join(base_path, 'merged_audio2.wav')


# 병합할 오디오 파일 목록 가져오기
audio_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.wav')]

# 병합할 오디오 데이터를 저장할 리스트 초기화
merged_audio_data = []
sample_rate = None

# 각 오디오 파일을 읽고 병합
for audio_file in tqdm(audio_files, desc="Merging audio files"):
    audio_data, sr = sf.read(audio_file)
    if sample_rate is None:
        sample_rate = sr  # 첫 번째 오디오 파일의 샘플링 레이트 사용
    else:
        assert sample_rate == sr, f"Sample rate mismatch: {audio_file}"
    merged_audio_data.append(audio_data)

# 병합된 오디오 데이터를 하나의 배열로 결합
if merged_audio_data:
    merged_audio_data = np.concatenate(merged_audio_data)
    # 병합된 오디오 데이터를 새로운 파일로 저장
    sf.write(merged_output_path, merged_audio_data, sample_rate)
    print(f"Merged audio file saved at {merged_output_path}")
else:
    print("No audio files to merge.")


Merging audio files: 100%|██████████| 2528/2528 [00:00<00:00, 4503.52it/s]


Merged audio file saved at /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/merged_audio2.wav


In [38]:
# 이 부분은 skip
# 00:19-00:20
# 00:30-00:37
# 14:23-14:25
# 28:12-28:13
# 29:00-29:04

import os
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

# 경로 설정
input_audio_path = os.path.join(base_path, 'merged_audio2.wav')
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_silent_split')

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 오디오 파일 읽기
processed_audio, sr = librosa.load(input_audio_path, sr=None)

noisy_datasets = []

i=0

# 5초씩 자르기
start = 0
segment_count = 0
for start in tqdm(range(0, len(processed_audio), 5 * sr), desc="Processing audio segments"):
    end = start + 5 * sr
    if end <= len(processed_audio):
        segment = processed_audio[start:end]
        output_audio_path = os.path.join(output_dir, f"UNLABELED_{i:05d}.wav")
        sf.write(output_audio_path, segment, sr)
        segment_count += 1

        i += 1
        noisy_datasets.append({
                'id1': audio_path,
                'id2': "",
                'path': output_audio_path,
                'fake': 0,
                'real': 0
            })
    else:
        # 마지막 segment가 5초 미만이면 저장하지 않음
        break

print("Processing completed.")


Processing audio segments: 100%|█████████▉| 259/260 [00:00<00:00, 288.05it/s]

Processing completed.


In [41]:

noisy_datasets = pd.DataFrame(noisy_datasets)
noisy_datasets.to_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', 'noisy_datasets.csv'), index=False)

In [49]:
base_path = os.getcwd()
train_df = pd.read_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', "train_bandstop_new_dataset_addmix.csv"))
test_df = pd.read_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', "test.csv"))

x_train_zero = train_df['path'].values.tolist()[-384:] * 156
y_train_zero = train_df[['fake', 'real']][-384:].values.tolist() * 156

x_train = train_df['path'][:-384].values.tolist() + x_train_zero
y_train = train_df[['fake', 'real']][:-384].values.tolist() + y_train_zero

x_test = [os.path.join(base_path, 'dataset', 'dacon_dataset', "test_bandstop_denoising", f"enhanced_{i}.wav") for i in test_df['id'].tolist()]

print(len(x_train), len(x_test))

239971 50000


In [50]:
label_mapping = {
    (1, 0): 0,
    (0, 1): 1,
    (2, 0): 2,
    (0, 2): 3,
    (1, 1): 4,
    (0, 0): 5
}

y_train_transformed = [label_mapping[tuple(labels)] for labels in y_train]



In [51]:
from sklearn.model_selection import train_test_split
_, X_val, _, y_val = train_test_split(x_train, y_train_transformed, test_size=0.005, stratify=y_train_transformed, random_state=42)

print(f"val: {len(X_val)}")

val: 1200


In [54]:
zero_zero_datasets = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_silent_split', f'{i}') for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'unlabeled_data_silent_split'))]


In [56]:
import numpy as np
import random
import soundfile as sf

def pad_and_randomize_audio(audio, sr=16000, target_length=5):
    """5초 이하의 음성을 5초 길이로 패딩하고 랜덤 위치에 배치"""
    target_samples = sr * target_length
    audio_length = len(audio)

    if audio_length >= target_samples:
        return audio[:target_samples]  # 이미 5초 이상인 경우, 잘라서 반환

    # 5초 이하인 경우
    padded_audio = np.zeros(target_samples)
    start_pos = random.randint(0, target_samples - audio_length)
    padded_audio[start_pos:start_pos + audio_length] = audio
    return padded_audio



In [57]:
import soundfile as sf
import librosa
import random

output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'valid')
os.makedirs(output_dir, exist_ok=True)
val_datasets = []

for i, (path, label) in tqdm(enumerate(zip(X_val, y_val)), total=len(X_val)):
    audio, sr = librosa.load(path)
    padded_audio = pad_and_randomize_audio(audio, sr, target_length=5)

    noisy_audio, noisy_sr = librosa.load(random.choice(zero_zero_datasets))
    audio = librosa.resample(padded_audio, orig_sr=sr, target_sr=noisy_sr)
    noisy_audio = noisy_audio[:noisy_sr*5]
    combined_audio = noisy_audio + audio

    noisy_audio, noisy_sr = librosa.load(random.choice(zero_zero_datasets))
    noisy_audio = noisy_audio[:noisy_sr*5]
    combined_audio += noisy_audio
    
    if random.random() < 0.7:
        noisy_audio, noisy_sr = librosa.load(random.choice(zero_zero_datasets))
        noisy_audio = noisy_audio[:noisy_sr*5]
        combined_audio += noisy_audio

    output_path = os.path.join(output_dir, f"VALID_{i:05d}.wav")
    val_datasets.append({"path": output_path, "label": label})
    sf.write(output_path, combined_audio, noisy_sr)

100%|██████████| 1200/1200 [00:18<00:00, 66.47it/s]


In [60]:
val_df = pd.DataFrame(val_datasets)
val_df.to_csv(os.path.join(base_path, 'valid.csv'), index=False)
 

In [62]:
import glob
import math
import os
import tempfile
import time
from typing import List, Optional, Tuple, Union

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import torch
from loguru import logger
from PIL import Image
from torch import Tensor
from torchaudio.backend.common import AudioMetaData

from df import config
from df.enhance import enhance, init_df, load_audio, save_audio
from df.io import resample

from torchaudio.backend.common import AudioMetaData

In [64]:
import torch
from tqdm.auto import tqdm
import shutil

import warnings
warnings.filterwarnings(action='ignore')

dir_name = 'valid_denoising'
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name)
os.makedirs(output_dir, exist_ok=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model, df, _ = init_df("./DeepFilterNet2", config_allow_defaults=True)
model = model.to(device=device).eval()
train_x = os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'valid'))


for data in tqdm(val_datasets):
    id = data['path'].split('/')[-1].split('.')[0]
    sr = config("sr", 32000, int, section="df")
    sample, meta = load_audio(data['path'], sr)
    enhanced = enhance(model, df, sample)

    lim = torch.linspace(0.0, 1.0, int(sr * 0.15)).unsqueeze(0)
    lim = torch.cat((lim, torch.ones(1, enhanced.shape[1] - lim.shape[1])), dim=1)
    enhanced = enhanced * lim
    if meta.sample_rate != sr:
        enhanced = resample(enhanced, sr, meta.sample_rate)
        sample = resample(sample, sr, meta.sample_rate)
        sr = meta.sample_rate

    noisy_wav = "./noisy_wav1551.wav"
    save_audio(noisy_wav, sample, sr)
    enhanced_wav = "./enhanced_wav1551.wav"
    save_audio(enhanced_wav, enhanced, sr)
    noisy_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f"noisy_{id}.wav")
    shutil.move("./noisy_wav1551.wav", noisy_wav)
    enhanced_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f"enhanced_{id}.wav")
    shutil.move("./enhanced_wav1551.wav",  enhanced_wav)


        

2024-07-23 11:37:46 | INFO     | DF | Loading model settings of DeepFilterNet2
2024-07-23 11:37:46 | INFO     | DF | Initializing model `deepfilternet2`
2024-07-23 11:37:46 | INFO     | DF | Found checkpoint ./DeepFilterNet2/checkpoints/model_96.ckpt.best with epoch 96
2024-07-23 11:37:46 | INFO     | DF | Running on device cuda:0
2024-07-23 11:37:46 | INFO     | DF | Model loaded


  0%|          | 0/1200 [00:00<?, ?it/s]